**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
# autoformatting
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import json
import io

import numpy as np
import base64
from IPython.display import HTML
import skvideo.io
import cv2

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import (
    Flatten,
    Activation,
    Conv2D,
)

<IPython.core.display.Javascript object>

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class EnvironmentAbstract(object):
    def __init__(self):
        raise NotImplementedError()

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        raise NotImplementedError()

    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        raise NotImplementedError()

    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        raise NotImplementedError()

<IPython.core.display.Javascript object>

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class AgentAbstract(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action

    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action)
            else:
                a = self.learned_act(s)
        else:  # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        raise NotImplementedError()

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        raise NotImplementedError()

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        raise NotImplementedError()

    def load(self):
        """ This function allows to restore a model"""
        raise NotImplementedError()

<IPython.core.display.Javascript object>

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

Here's what the function `act` does:

- during the training phase:

    - with probability `epsilon`, we take a random action
    - with probability `1 - epsilon`, we used our current policy to take the "best" possible action

- during the evaluation phase, we always use our trained policy to take the best possible action.


`epsilon` is essential because it gives us control over the exploration/exploitation trade-off during the training phase:

- exploration: taking possibly non-optimal actions to explore new ways of interacting with the environment, hopefully discovering better ways of getting reward in the process

- exploitation: using our current policy to take what the agent believes is the best action in order to maximize its reward''

If `epsilon` is too high, the agent won't train, because it won't take enough good actions to discover the optimal strategy (which is a succession of good actions).

If `epsilon` is too low, the agent won't train either, because it will be heavily biased towards its initial policy and won't explore enough new ways of interacting with the world to discover the optimal strategy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(EnvironmentAbstract):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros(
            (max_time + 2, grid_size * self.scale, grid_size * self.scale, 3)
        )

    def draw(self, e):
        skvideo.io.vwrite(str(e) + ".mp4", self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0

        b = cv2.resize(
            b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST
        )

        self.to_draw[t, :, :, :] = b

    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError("Error: action not recognized")

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate(
            (
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )
        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size - 3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size - 3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros(
            (
                self.max_time + 2,
                self.grid_size * self.scale,
                self.grid_size * self.scale,
                3,
            )
        )

        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate(
            (
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )

        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]
        return state

<IPython.core.display.Javascript object>

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# display videos
def display_videos(name):
    video = io.open(name, "r+b").read()
    encoded = base64.b64encode(video)
    return """<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>""".format(
        encoded.decode("ascii")
    )

<IPython.core.display.Javascript object>

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

*Part 1*

`position[x,y]` gives us some information about position `x,y`:
    
- `1`: rat position
- `0`: empty square (ie, no wall)
- `-1`: wall
    
A window of the `position` array is visible by the agent, as it's part of the state. The agent can use this part of the state to figure out that walking into walls is probably not a good idea.

*Part 2*

`board` is used to keep track of the rewards (positive or negative) that are still left on the board, `board[x,y]` can be any value in `[-1, 0.5, 0]`, the the rat walks on a square, it wins the reward stored in `board[x,y]` and then the value of `board[x,y]` is set to `0`.

Like the `position` array, a window of the `board` array is visible by the agent.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(AgentAbstract):
    def __init__(self):
        super().__init__()

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

<IPython.core.display.Javascript object>

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent, env, epochs, prefix=""):
    # Number of won games
    score = 0

    for e in range(epochs):

        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print(
            "Win/lose count {}/{}. Average score ({})".format(
                win, lose, score / (1 + e)
            )
        )
    print("Final score: " + str(score / epochs))

<IPython.core.display.Javascript object>

In [9]:
size = 13
T = 200
temperature = 0.3
epochs_test = 15

# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix="video/random")
HTML(display_videos("video/random0.mp4"))

Win/lose count 9.0/21.0. Average score (-12.0)
Win/lose count 8.0/7.0. Average score (-5.5)
Win/lose count 8.5/15.0. Average score (-5.833333333333333)
Win/lose count 7.0/10.0. Average score (-5.125)
Win/lose count 9.0/13.0. Average score (-4.9)
Win/lose count 6.5/12.0. Average score (-5.0)
Win/lose count 8.5/9.0. Average score (-4.357142857142857)
Win/lose count 10.0/21.0. Average score (-5.1875)
Win/lose count 8.5/13.0. Average score (-5.111111111111111)
Win/lose count 18.0/12.0. Average score (-4.0)
Win/lose count 12.0/13.0. Average score (-3.727272727272727)
Win/lose count 10.0/22.0. Average score (-4.416666666666667)
Win/lose count 10.5/14.0. Average score (-4.346153846153846)
Win/lose count 8.5/13.0. Average score (-4.357142857142857)
Win/lose count 9.0/10.0. Average score (-4.133333333333334)
Final score: -4.133333333333334


<IPython.core.display.Javascript object>

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Question 5 (anwser)**

*Part 1*

$$
\begin{align*}
Q^\pi(s, a) & = E_{p^\pi}[\sum_{t \geq 0} \gamma^t r(s_t, a_t) | s_0=s, a_0=a] \\
& = E_{p^\pi(.|s,a)}[\sum_{t \geq 0} \gamma^t r(s_t, a_t)] \\
& = E_{p^\pi(.|s,a)}[r(s, a) + \gamma E_{p^\pi}[\sum_{t \geq 0} \gamma^t r(s_{t + 1}, a_{t + 1})|s_1, a_1]] \text{ because }\{(a_t, s_t)\}_{t \geq 0}\text{ is a Markov process} \\
& = E_{(a', s') \sim p^\pi(.|s,a)}[r(s, a) + \gamma Q^\pi(s', a')] \\
\end{align*}
$$

<!-- By definition:

$$
Q^*(s,a) = \max_\pi Q^\pi(s,a)
$$

And we have:

$$
\pi^*(s) = \operatorname{argmax}_a Q^*(s,a)
$$

The previous equation is valid for $Q^*$, thus we get:

$$
\begin{align*}
Q^*(s, a) & = E_{(a', s') \sim \pi^*(.|s,a)}[r(s, a) + \gamma Q^*(s', a')] \\
& = E_{s' \sim p(.|s,a)}[r(s, a) + \gamma \max_{a'} Q^*(s', a')] \\
\end{align*}
$$ -->

*Part 2*

By definition of $Q^*$:

$$
\begin{align*}
Q^*(s,a) & = \max_\pi Q^\pi(s,a) \\
& = \max_\pi E_{(a', s') \sim p(.|s,a)}[r(s, a) + \gamma Q^\pi(s', a')] \\
& = r(s, a) + \max_\pi \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s') \gamma Q^\pi(s', a') \\
& \leq r(s, a) + \sum_{s'} p(s'|s,a) \max_\pi \sum_{a'} \pi(a'|s') \gamma Q^\pi(s', a') \\
& \leq r(s, a) + \sum_{s'} p(s'|s,a) \max_{\pi, a'} \gamma Q^\pi(s', a') \\
\end{align*}
$$

Let:

$$
\begin{align*}
\pi_0(.|s) & = arg \max_\pi \max_a Q^\pi(s, a) \\
\pi_1(a|s) & = \mathbb{1}_{a = \pi_0(a|s)} \\
\end{align*}
$$

The last inequality can be rewritten (by definition of $\pi_1$):

$$
\begin{align*}
Q^*(s,a) & \leq Q^{\pi_1}(s,a) = r(s, a) + \sum_{s'} p(s'|s,a) \sum_{a'} \pi_1(a'|s') \gamma Q^{\pi_1}(s', a')
\end{align*}
$$

But, $\pi_1$ being a policy, we also have (by definition of $Q^*$):

$$
Q^{\pi_1}(s,a) \leq Q^*(s,a)
$$

Thus:

$$
Q^{\pi_1}(s,a) = Q^*(s,a)
$$

*Part 3*

Our objective is to find a $Q^*$ that verifies the previous equation, which we can rewrite:

$$
\begin{align*}
& E_{s' \sim \pi^*(.|s,a)}[r(s, a) + \max_{a'} \gamma Q^*(s', a')] - Q^*(s, a) & = 0 \\
& E_{s' \sim \pi^*(.|s,a)}[r(s, a) + \max_{a'} \gamma Q^*(s', a') - Q^*(s, a)] & = 0 \\
\text{which is implied by } & E_{s' \sim \pi^*(.|s,a)}[\|r(s, a) + \max_{a'} \gamma Q^*(s', a') - Q^*(s, a)\|^2] & = 0
\end{align*}
$$

Minimizing the following function will thus allow us to find the optimal $Q^*$:

$$
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}[\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}]
$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()
        self.cursor = 0  # if the memory is full, which index should we overwrite ?

    def remember(self, m):
        # if we have room for the memory to grow, make it grow
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        # otherwise, overwrite memory[cursor]
        else:
            self.memory[self.cursor] = m
            self.cursor = (self.cursor + 1) % len(self.memory)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

<IPython.core.display.Javascript object>

***
The pipeline we will use for training is given below:

In [11]:
def train(agent, env, epoch, prefix=""):
    # Number of won games
    score = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        loss = 0
        turns = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss += agent.reinforce(prev_state, state, action, reward, game_over)
            turns += 1

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print(
            "Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})".format(
                e, epoch, loss / turns, win, lose, win - lose
            )
        )
        agent.save(name_weights=prefix + "model.h5", name_model=prefix + "model.json")

<IPython.core.display.Javascript object>

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQNAbstract(AgentAbstract):
    def __init__(
        self, grid_size, epsilon=0.1, memory_size=100, batch_size=16, n_state=2
    ):
        super().__init__(epsilon=epsilon)

        # Q-learning discount:
        # I chose a value that's smaller than the original value (0.99) to make
        # sure the Q-values won't diverge
        self.discount = 0.8

        self.grid_size = grid_size

        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)

        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[np.newaxis]), 1)[0]

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])

        batch = [self.memory.random_access() for _ in range(self.batch_size)]

        states = np.array([e[0] for e in batch])
        next_states = np.array([e[1] for e in batch])
        actions = np.array([e[2] for e in batch])
        rewards = np.array([e[3] for e in batch])
        game_overs = np.array([e[4] for e in batch])

        input_states = states
        target_q = self.model.predict(states)
        target_q_next = self.model.predict(next_states)

        target_q[
            np.where(game_overs == 1), actions[np.where(game_overs == 1)]
        ] = rewards[game_overs == 1]
        target_q[
            np.where(game_overs == 0), actions[np.where(game_overs == 0)]
        ] = rewards[game_overs == 0] + self.discount * np.max(
            target_q_next[game_overs == 0]
        )

        # with a discount factor of 0.8, there is no need of clipping
        # the target Q-values, that's why the following line is commented out

        # target_q = np.clip(target_q, -5, 5)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self, name_weights="model.h5", name_model="model.json"):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)

    def load(self, name_weights="model.h5", name_model="model.json"):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model


class DQN_FC(DQNAbstract):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_FC, self).__init__(*args, **kwargs)

        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(30, activation="relu"))
        model.add(Dense(4))

        model.compile(SGD(lr=lr, decay=1e-4, momentum=0.0), "mse")

        self.model = model

<IPython.core.display.Javascript object>

In [13]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 50

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent.model.summary()
train(agent, env, epochs_train, prefix="video/fc_train")
HTML(display_videos("video/fc_train50.mp4"))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 30)                1530      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 124       
Total params: 1,654
Trainable params: 1,654
Non-trainable params: 0
_________________________________________________________________
Epoch 001/050 | Loss 0.0406 | Win/lose count 2.5/4.0 (-1.5)
Epoch 002/050 | Loss 0.0149 | Win/lose count 2.0/0 (2.0)
Epoch 003/050 | Loss 0.0146 | Win/lose count 1.0/4.0 (-3.0)
Epoch 004/050 | Loss 0.0164 | Win/lose count 3.0/7.0 (-4.0)
Epoch 005/050 | Loss 0.0179 | Win/lose count 4.0/11.0 (-7.0)
Epoch 006/050 | Loss 0.0143 | Win/lose count 2.0/5.0 (-3.0)
Epoch 007/050 | Loss 0.012

<IPython.core.display.Javascript object>

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQNAbstract):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_CNN, self).__init__(*args, **kwargs)

        model = Sequential()
        model.add(
            Conv2D(32, (3, 3), input_shape=(5, 5, self.n_state), activation="relu")
        )
        model.add(Conv2D(16, (3, 3), activation="relu"))
        model.add(Flatten())
        model.add(Dense(4))

        model.compile(SGD(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

<IPython.core.display.Javascript object>

In [15]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 50

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent.model.summary()
train(agent, env, epochs_train, prefix="video/cnn_train")
HTML(display_videos("video/cnn_train50.mp4"))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 3, 32)          608       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 16)          4624      
_________________________________________________________________
flatten_1 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
Total params: 5,300
Trainable params: 5,300
Non-trainable params: 0
_________________________________________________________________
Epoch 001/050 | Loss 0.0141 | Win/lose count 2.0/3.0 (-1.0)
Epoch 002/050 | Loss 0.0093 | Win/lose count 1.5/1.0 (0.5)
Epoch 003/050 | Loss 0.0046 | Win/lose count 1.5/2.0 (-0.5)
Epoch 004/050 | Loss 0.0042 | Win/lose count 5.5/3.0 (2.5)
Epoch 005/05

<IPython.core.display.Javascript object>

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
# parameters
epochs_test = 10
temperature = 0.5

env = Environment(grid_size=size, max_time=T, temperature=temperature)

agent_cnn = DQN_CNN(size, lr=0.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent_cnn.load(
    name_weights="video/cnn_trainmodel.h5", name_model="video/cnn_trainmodel.json"
)

agent_fc = DQN_FC(size, lr=0.1, epsilon=0.1, memory_size=2000, batch_size=32)
agent_fc.load(
    name_weights="video/fc_trainmodel.h5", name_model="video/fc_trainmodel.json"
)

print("Test of the CNN")
test(agent_cnn, env, epochs_test, prefix="video/cnn_test")

print("Test of the FC")
test(agent_fc, env, epochs_test, prefix="video/fc_test")

Test of the CNN
Win/lose count 14.5/0. Average score (14.5)
Win/lose count 9.5/0. Average score (12.0)
Win/lose count 2.5/0. Average score (8.833333333333334)
Win/lose count 3.5/0. Average score (7.5)
Win/lose count 3.5/0. Average score (6.7)
Win/lose count 7.0/0. Average score (6.75)
Win/lose count 1.5/0. Average score (6.0)
Win/lose count 9.5/0. Average score (6.4375)
Win/lose count 5.5/1.0. Average score (6.222222222222222)
Win/lose count 5.0/0. Average score (6.1)
Final score: 6.1
Test of the FC
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 4.0/0. Average score (2.3333333333333335)
Win/lose count 6.5/1.0. Average score (3.125)
Win/lose count 7.0/0. Average score (3.9)
Win/lose count 6.5/1.0. Average score (4.166666666666667)
Win/lose count 19.5/0. Average score (6.357142857142857)
Win/lose count 16.0/0. Average score (7.5625)
Win/lose count 2.0/0. Average score (6.944444444444445)
Win/lose count 21.5/0. Average score (8.4)
Final 

<IPython.core.display.Javascript object>

In [18]:
HTML(display_videos("video/cnn_test0.mp4"))

<IPython.core.display.Javascript object>

In [19]:
HTML(display_videos("video/fc_test0.mp4"))

<IPython.core.display.Javascript object>

The fully-connected model seems to perform better than the convolutional model.

I observe that once the model has eaten all the cheese that was in his visible space, it tends to stay where it currently is without doing anything other than walking bath and forth between two squares.

We can "fix" this behavior by changing the game: if we increase the temperature, there is more cheese on the board, and thus, the agent will tend to discover new pieces of cheese when walking towards the pieces of cheese it can see.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [20]:
def train_explore(agent, env, epoch, prefix=""):
    # Number of won games
    score = 0

    epsilon_start = agent.epsilon
    epsilon_end = 0.1

    for e in range(1, epoch + 1):
        state = env.reset()
        game_over = False

        # exponentially decreasing epsilon
        agent.set_epsilon(
            epsilon_end + (epsilon_start - epsilon_end) * np.exp(-(e - 1) / 10)
        )

        win = 0
        lose = 0

        loss = 0
        turns = 0

        while not game_over:

            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss += agent.reinforce(prev_state, state, action, reward, game_over)
            turns += 1

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print(
            "Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.1f}) | Epsilon {:.4f}".format(
                e, epoch, loss / turns, win, lose, win - lose, agent.epsilon
            )
        )
        agent.save(name_weights=prefix + "model.h5", name_model=prefix + "model.json")


class EnvironmentExploring(EnvironmentAbstract):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))
        self.malus_position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros(
            (max_time + 2, grid_size * self.scale, grid_size * self.scale, 3)
        )

    def draw(self, e):
        skvideo.io.vwrite(str(e) + ".mp4", self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0

        b = cv2.resize(
            b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST
        )

        self.to_draw[t, :, :, :] = b

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError("Error: action not recognized")

        self.t = self.t + 1

        # during train, reward is now a sum of the "real" reward and a "fake" term that
        # encourages exploration
        reward = (
            self.board[self.x, self.y] - train * self.malus_position[self.x, self.y]
        )

        self.board[self.x, self.y] = 0

        # at every visit of [x,y], we increase the malus by 0.1, but we keep it below
        # 0.5 because we want re-visiting a place to be cheaper that eating a real malus
        self.malus_position[self.x, self.y] += 0.1
        self.malus_position = np.clip(self.malus_position, a_min=0, a_max=0.4)

        game_over = self.t > self.max_time

        state = np.concatenate(
            (
                self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )
        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size - 3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size - 3, size=1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros(
            (
                self.max_time + 2,
                self.grid_size * self.scale,
                self.grid_size * self.scale,
                3,
            )
        )

        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate(
            (
                self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size, 1),
            ),
            axis=2,
        )

        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]
        return state

<IPython.core.display.Javascript object>

In [21]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 100

# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=0.1, epsilon=0.95, memory_size=2000, batch_size=32, n_state=3)
agent.model.summary()
train_explore(agent, env, epochs_train, prefix="video/cnn_train_explore")
HTML(display_videos("video/cnn_train_explore10.mp4"))

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 3, 32)          896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 1, 1, 16)          4624      
_________________________________________________________________
flatten_5 (Flatten)          (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 4)                 68        
Total params: 5,588
Trainable params: 5,588
Non-trainable params: 0
_________________________________________________________________
Epoch 001/100 | Loss 0.0323 | Win/lose count 12.5/43.0 (-30.5) | Epsilon 0.9500
Epoch 002/100 | Loss 0.0135 | Win/lose count 8.5/44.2 (-35.7) | Epsilon 0.8691
Epoch 003/100 | Loss 0.0133 | Win/lose count 9.5/42.6 (-33.1) | Epsilon 0.7959
Epoch 004/10

Epoch 094/100 | Loss 0.0057 | Win/lose count 19.5/21.9 (-2.4) | Epsilon 0.1001
Epoch 095/100 | Loss 0.0081 | Win/lose count 17.5/14.8 (2.7) | Epsilon 0.1001
Epoch 096/100 | Loss 0.0073 | Win/lose count 16.5/21.9 (-5.4) | Epsilon 0.1001
Epoch 097/100 | Loss 0.0079 | Win/lose count 13.5/25.8 (-12.3) | Epsilon 0.1001
Epoch 098/100 | Loss 0.0068 | Win/lose count 25.0/11.7 (13.3) | Epsilon 0.1001
Epoch 099/100 | Loss 0.0062 | Win/lose count 23.5/14.2 (9.3) | Epsilon 0.1000
Epoch 100/100 | Loss 0.0072 | Win/lose count 18.5/16.2 (2.3) | Epsilon 0.1000


<IPython.core.display.Javascript object>

In [25]:
# Evaluation
epochs_test = 15

env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent, env, epochs_test, prefix="video/cnn_test_explore")
HTML(display_videos("video/cnn_test_explore0.mp4"))

Win/lose count 23.5/1.0. Average score (22.5)
Win/lose count 25.0/0. Average score (23.75)
Win/lose count 23.5/0. Average score (23.666666666666668)
Win/lose count 19.5/0. Average score (22.625)
Win/lose count 23.0/0. Average score (22.7)
Win/lose count 20.0/0. Average score (22.25)
Win/lose count 25.5/0. Average score (22.714285714285715)
Win/lose count 21.0/1.0. Average score (22.375)
Win/lose count 17.0/0. Average score (21.77777777777778)
Win/lose count 14.5/0. Average score (21.05)
Win/lose count 18.5/1.0. Average score (20.727272727272727)
Win/lose count 22.0/0. Average score (20.833333333333332)
Win/lose count 12.0/0. Average score (20.153846153846153)
Win/lose count 9.0/0. Average score (19.357142857142858)
Win/lose count 16.0/1.0. Average score (19.066666666666666)
Final score: 19.066666666666666


<IPython.core.display.Javascript object>

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***